Perform text cleaning, perform lemmatization (any method), remove stop words (any method), label encoding. 

Create representations using TF-IDF. 

Save outputs.

Dataset: https://github.com/PICT-NLP/BE-NLP-Elective/blob/main/3-Preprocessing/News_dataset.pickle

In [1]:
#Importing Libraries
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

In [4]:
path_df = "/content/News_dataset.pickle"
with open(path_df, 'rb') as data:
    df = pickle.load(data)

In [5]:
#checking data
df.head()

File_Name                                            Content  Category  \
0   001.txt  Ad sales boost Time Warner profit\r\n\r\nQuart...  business   
1   002.txt  Dollar gains on Greenspan speech\r\n\r\nThe do...  business   
2   003.txt  Yukos unit buyer faces loan claim\r\n\r\nThe o...  business   
3   004.txt  High fuel prices hit BA's profits\r\n\r\nBriti...  business   
4   005.txt  Pernod takeover talk lifts Domecq\r\n\r\nShare...  business   

  Complete_Filename  id  News_length  
0  001.txt-business   1         2569  
1  002.txt-business   1         2257  
2  003.txt-business   1         1557  
3  004.txt-business   1         2421  
4  005.txt-business   1         1575

In [6]:
#Chcking article
df.loc[1]['Content']

'Dollar gains on Greenspan speech\r\n\r\nThe dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise.\r\n\r\nAnd Alan Greenspan highlighted the US government\'s willingness to curb spending and rising household savings as factors which may help to reduce it. In late trading in New York, the dollar reached $1.2871 against the euro, from $1.2974 on Thursday. Market concerns about the deficit has hit the greenback in recent months. On Friday, Federal Reserve chairman Mr Greenspan\'s speech in London ahead of the meeting of G7 finance ministers sent the dollar higher after it had earlier tumbled on the back of worse-than-expected US jobs data. "I think the chairman\'s taking a much more sanguine view on the current account deficit than he\'s taken for some time," said Robert Sinche, head of currency strategy at Bank of America in New York. "He\'s taking a longer-term view, laying out a set of cond

###Text cleaning and preparation


In [7]:
#Text cleaning

df['Content_Parsed_1'] = df['Content'].str.replace("\r", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')

In [8]:
#Text preparation

df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()         #all to lower case

punctuation_signs = list("?:!.,;")                                  #remove punctuations
df['Content_Parsed_3'] = df['Content_Parsed_2']

for punct_sign in punctuation_signs:
    df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')

df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")       #remove possessive pronouns

<ipython-input-8-3fcdc84e92bf>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')


###Lemmatization

In [9]:
nltk.download('punkt')
nltk.download('wordnet')

nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [10]:
wordnet_lemmatizer = WordNetLemmatizer()
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

df['Content_Parsed_5'] = lemmatized_text_list  
df['Content_Parsed_5']  

0       ad sales boost time warner profit quarterly pr...
1       dollar gain on greenspan speech the dollar hav...
2       yukos unit buyer face loan claim the owners of...
3       high fuel price hit ba profit british airways ...
4       pernod takeover talk lift domecq share in uk d...
                              ...                        
2220    bt program to beat dialler scam bt be introduc...
2221    spam e-mail tempt net shoppers computer users ...
2222    be careful how you code a new european directi...
2223    us cyber security chief resign the man make su...
2224    lose yourself in online game online role play ...
Name: Content_Parsed_5, Length: 2225, dtype: object

###Removing stop words

In [11]:
nltk.download('stopwords')
stop_words = list(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
df['Content_Parsed_6'] = df['Content_Parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')
df.loc[5]['Content_Parsed_6']

<ipython-input-12-1979480f87d6>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')


'japan narrowly escape recession japan economy teeter   brink   technical recession   three months  september figure show revise figure indicate growth   01% -   similar-sized contraction   previous quarter   annual basis  data suggest annual growth   02% suggest  much  hesitant recovery   previously  think  common technical definition   recession  two successive quarter  negative growth  government  keen  play   worry implications   data  maintain  view  japan economy remain   minor adjustment phase   upward climb    monitor developments carefully say economy minister heizo takenaka    face   strengthen yen make export less competitive  indications  weaken economic condition ahead observers  less sanguine  paint  picture   recovery much patchier  previously think say paul sheard economist  lehman brothers  tokyo improvements   job market apparently  yet  fee   domestic demand  private consumption   02%   third quarter'

In [13]:

#Checking data

df.head(1)

File_Name                                            Content  Category  \
0   001.txt  Ad sales boost Time Warner profit\r\n\r\nQuart...  business   

  Complete_Filename  id  News_length  \
0  001.txt-business   1         2569   

                                    Content_Parsed_1  \
0  Ad sales boost Time Warner profit Quarterly pr...   

                                    Content_Parsed_2  \
0  ad sales boost time warner profit quarterly pr...   

                                    Content_Parsed_3  \
0  ad sales boost time warner profit quarterly pr...   

                                    Content_Parsed_4  \
0  ad sales boost time warner profit quarterly pr...   

                                    Content_Parsed_5  \
0  ad sales boost time warner profit quarterly pr...   

                                    Content_Parsed_6  
0  ad sales boost time warner profit quarterly pr...

In [14]:
#Removing the old content_parsed columns

list_columns = ["File_Name", "Category", "Complete_Filename", "Content", "Content_Parsed_6"]
df = df[list_columns]

df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})

##Label coding

In [19]:
#Generating new column for Category codes
category_codes = {
    'business': 0,
    'entertainment': 1,
    'politics': 2,
    'sport': 3,
    'tech': 4
}

# Category mapping
df['Category_Code'] = df['Category']
df = df.replace({'Category_Code':category_codes})
df.head()

File_Name  Category Complete_Filename  \
0   001.txt  business  001.txt-business   
1   002.txt  business  002.txt-business   
2   003.txt  business  003.txt-business   
3   004.txt  business  004.txt-business   
4   005.txt  business  005.txt-business   

                                             Content  \
0  Ad sales boost Time Warner profit\r\n\r\nQuart...   
1  Dollar gains on Greenspan speech\r\n\r\nThe do...   
2  Yukos unit buyer faces loan claim\r\n\r\nThe o...   
3  High fuel prices hit BA's profits\r\n\r\nBriti...   
4  Pernod takeover talk lifts Domecq\r\n\r\nShare...   

                                      Content_Parsed  Category_Code  
0  ad sales boost time warner profit quarterly pr...              0  
1  dollar gain  greenspan speech  dollar  hit  hi...              0  
2  yukos unit buyer face loan claim  owners  emba...              0  
3  high fuel price hit ba profit british airways ...              0  
4  pernod takeover talk lift domecq share  uk dri...              0

In [16]:
#Train - test split
X_train, X_test, y_train, y_test = train_test_split(df['Content_Parsed'], 
                                                    df['Category_Code'], 
                                                    test_size=0.15, 
                                                    random_state=8)

In [26]:
vectorizer = TfidfVectorizer()
                        
features_train = vectorizer.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = vectorizer.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(1891, 23976)
(334, 23976)


In [28]:
feature_names = list(vectorizer.vocabulary_.keys())
print("Feature names:", feature_names)

Feature names: ['bt', 'boost', 'broadband', 'package', 'british', 'telecom', 'say', 'double', 'speed', 'home', 'business', 'customers', 'increase', 'come', 'extra', 'charge', 'follow', 'similar', 'move', 'internet', 'service', 'provider', 'aol', 'many', 'download', '2mbps', 'although', 'usage', 'allowance', 'one', 'gigabyte', '30', 'gigabytes', 'month', 'new', 'start', 'effect', '17', 'february', 'april', 'businesses', 'britain', 'duncan', 'ingram', 'manage', 'director', 'add', 'ninety', 'percent', 'see', 'real', 'give', 'people', 'opportunity', 'lot', 'connections', 'upload', 'information', 'send', 'pc', 'via', 'remain', 'mr', 'despite', 'continue', 'extremely', 'generous', 'market', 'place', 'really', 'issue', 'begin', 'enforce', 'summer', 'exceed', 'amount', 'either', 'able', 'pay', 'bigger', 'reduce', '36', 'share', '39', 'become', 'increasingly', 'competitive', 'last', 'months', 'rival', 'isps', 'offer', 'include', 'plusnet', 'uk', 'online', 'lag', 'behind', 'countries', 'especial